In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv('../data/sculpture_dataset_labeled.csv')

df.head()

,filename,page,group,era
0,page184_img01_photo2.jpg,184,FORERUNNERS,Before 615 BC
1,page184_img01_photo3.jpg,184,FORERUNNERS,Before 615 BC
2,page184_img01_photo4.jpg,184,FORERUNNERS,Before 615 BC
3,page184_img01_photo6.jpg,184,FORERUNNERS,Before 615 BC
4,page184_img01_photo7.jpg,184,FORERUNNERS,Before 615 BC


In [ ]:
import torch
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from PIL import Image
import os
import timm
from tqdm.notebook import tqdm # Import tqdm for progress bar

In [ ]:
model_name = 'convnextv2_tiny' 
model = timm.create_model(model_name, pretrained=True)
model.eval()  # set to eval mode
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [4]:
batch_size = 32
embeddings = []

image_directory = "../data/richter_kouroi_filtered_photos"

# Collect filenames and eras for batching
filenames = df['filename'].tolist()
eras = df['era'].tolist()

for i in tqdm(range(0, len(filenames), batch_size), desc="Processing Images in Batches"):
    batch_filenames = filenames[i:i + batch_size]

    images = []
    valid_indices = []
    for j, filename in enumerate(batch_filenames):
        image_path = os.path.join(image_directory, filename)
        try:
            image = Image.open(image_path).convert('RGB')  # convert to RGB to avoid issues
            images.append(image)
            valid_indices.append(i + j)  # Keep track of valid indices for later
        except FileNotFoundError:
            print(f"Image not found at: {image_path}")
        except Exception as e:
            print(f"Error processing image {filename}: {e}")

    if len(images) == 0:
        continue  # Skip empty batches

    # Process the batch of images
    inputs = processor(images=images, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state

    # Mean pooling over patches per image in batch
    batch_embeddings = last_hidden_states.mean(dim=1).cpu().numpy()

    # Append embeddings maintaining order corresponding to valid images
    embeddings.extend(batch_embeddings)

Processing Images in Batches:   0%|          | 0/21 [00:00<?, ?it/s]

In [7]:
np.array(embeddings)

np.save('../data/embeddings/all_photos_embeddings.npy', embeddings)
np.save('../data/embeddings/all_photos_eras.npy', eras)

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X = np.load('../data/embeddings/all_photos_embeddings.npy')
y = np.load('../data/embeddings/all_photos_eras.npy')

# X = embeddings, y = your labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.5151515151515151


In [6]:
df = pd.read_csv('../data/complete_sculpture_dataset_labeled.csv')

batch_size = 32
embeddings = []

image_directory = "../data/richter_kouroi_complete_front_only"

# Collect filenames and eras for batching
filenames = df['filename'].tolist()
eras = df['era'].tolist()
material = df['material'].tolist()

for i in tqdm(range(0, len(filenames), batch_size), desc="Processing Images in Batches"):
    batch_filenames = filenames[i:i + batch_size]

    images = []
    valid_indices = []
    for j, filename in enumerate(batch_filenames):
        image_path = os.path.join(image_directory, filename)
        try:
            image = Image.open(image_path).convert('RGB')  # convert to RGB to avoid issues
            images.append(image)
            valid_indices.append(i + j)  # Keep track of valid indices for later
        except FileNotFoundError:
            print(f"Image not found at: {image_path}")
        except Exception as e:
            print(f"Error processing image {filename}: {e}")

    if len(images) == 0:
        continue  # Skip empty batches

    # Process the batch of images
    inputs = processor(images=images, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state

    # Mean pooling over patches per image in batch
    batch_embeddings = last_hidden_states.mean(dim=1).cpu().numpy()

    # Append embeddings maintaining order corresponding to valid images
    embeddings.extend(batch_embeddings)

Processing Images in Batches:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
np.array(embeddings)

np.save('../data/embeddings/complete_sculptures_embeddings.npy', embeddings)
np.save('../data/embeddings/complete_sculptures_eras.npy', eras)
np.save('../data/embeddings/complete_sculptures_material.npy', material)

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X = embeddings
y1 = eras
y2 = material

# X = embeddings, y = your labels
X_train, X_test, y_train, y_test = train_test_split(X, y1, test_size=0.2, random_state=42)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.3076923076923077


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y2, test_size=0.2, random_state=42)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.8461538461538461


In [10]:
df = pd.read_csv('../data/head_dataset_labeled.csv')

batch_size = 32
embeddings = []

image_directory = "../data/richter_kouroi_head_front_only"

# Collect filenames and eras for batching
filenames = df['filename'].tolist()
eras = df['era'].tolist()

for i in tqdm(range(0, len(filenames), batch_size), desc="Processing Images in Batches"):
    batch_filenames = filenames[i:i + batch_size]

    images = []
    valid_indices = []
    for j, filename in enumerate(batch_filenames):
        image_path = os.path.join(image_directory, filename)
        try:
            image = Image.open(image_path).convert('RGB')  # convert to RGB to avoid issues
            images.append(image)
            valid_indices.append(i + j)  # Keep track of valid indices for later
        except FileNotFoundError:
            print(f"Image not found at: {image_path}")
        except Exception as e:
            print(f"Error processing image {filename}: {e}")

    if len(images) == 0:
        continue  # Skip empty batches

    # Process the batch of images
    inputs = processor(images=images, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state

    # Mean pooling over patches per image in batch
    batch_embeddings = last_hidden_states.mean(dim=1).cpu().numpy()

    # Append embeddings maintaining order corresponding to valid images
    embeddings.extend(batch_embeddings)

Processing Images in Batches:   0%|          | 0/3 [00:00<?, ?it/s]

In [11]:
np.array(embeddings)

np.save('../data/embeddings/head_embeddings.npy', embeddings)
np.save('../data/embeddings/head_eras.npy', eras)

In [12]:
X = embeddings
y1 = eras

# X = embeddings, y = your labels
X_train, X_test, y_train, y_test = train_test_split(X, y1, test_size=0.2, random_state=42)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.5555555555555556
